In [12]:
import pandas as pd
import numpy as np

import tqdm

## Data load

In [13]:
data_path = '../data/'
train = pd.read_csv(data_path + 'train_data.csv')
test = pd.read_csv(data_path + 'test_data.csv')

In [14]:
train['grade'] = 1
test['grade'] = 2

In [15]:
len(train), len(test)

(2266586, 260114)

In [16]:
train['answerCode'].unique(), test['answerCode'].unique()

(array([1, 0], dtype=int64), array([ 1,  0, -1], dtype=int64))

### test에서 answerCode -1인 row 처리
마지막 문제 제외한 sequence까지 학습에 사용하기 위해서

inference에서만 붙여서 쓰기??

In [17]:
remove_index = list(test[test['answerCode']==-1].index)
len(remove_index)

744

In [18]:
test.loc[test[test['answerCode']==-1].index, 'grade'] = 3

In [19]:
# test = test.drop(remove_index)
# len(test), test['answerCode'].unique()

In [20]:
data = pd.concat((train, test), axis=0)
data = data.sort_values('userID')

In [21]:
# total data
data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
491,0,A060099007,A060000099,1,2020-09-03 01:08:58,705,1
492,0,A080082001,A080000082,1,2020-09-05 01:08:00,30,1
493,0,A080082002,A080000082,1,2020-09-05 01:09:20,32,1
494,0,A080082003,A080000082,0,2020-09-05 01:09:55,32,1
...,...,...,...,...,...,...,...
2266579,7441,A030071003,A030000071,1,2020-06-05 06:49:23,438,1
2266578,7441,A030071002,A030000071,0,2020-06-05 06:48:43,438,1
2266577,7441,A030071001,A030000071,0,2020-06-05 06:46:41,438,1
2266580,7441,A030071004,A030000071,0,2020-06-05 06:49:57,438,1


In [22]:
# total data 저장
data.to_csv(data_path + 'total_data.csv', index=False)

In [23]:
user_id = list(data['userID'].unique())

### user별 각 class의 count, acc, elapsetime
class : 대분류(시험지 카테고리), 중분류(시험지번호), 소분류(문항번호)

In [24]:
data = pd.read_csv(data_path + 'total_data.csv', parse_dates=["Timestamp"])

In [25]:
data.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
1,0,A060099007,A060000099,1,2020-09-03 01:08:58,705,1
2,0,A080082001,A080000082,1,2020-09-05 01:08:00,30,1
3,0,A080082002,A080000082,1,2020-09-05 01:09:20,32,1
4,0,A080082003,A080000082,0,2020-09-05 01:09:55,32,1


### elapse time
직전 timestamp와의 차이 -> 현재 row에 직전문제의 풀이시간 추가

In [26]:
data = data.sort_values(by=['userID','Timestamp']).reset_index(drop=True)
data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1
...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1


In [27]:
elapsed = data.loc[:,['userID','Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
elapsed = elapsed.fillna(pd.Timedelta(seconds=0))
elapsed = elapsed['Timestamp'].apply(lambda x: x.total_seconds())
elapsed

0                0.0
1                3.0
2                8.0
3                7.0
4                7.0
             ...    
2526695         24.0
2526696    6632178.0
2526697         11.0
2526698         46.0
2526699         73.0
Name: Timestamp, Length: 2526700, dtype: float64

In [28]:
data['elapsed'] = elapsed
data['elapsed'] = data['elapsed'].apply(lambda x: x if x < 600 and x>=0 else 0)
data.head(10)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade,elapsed
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,3.0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,8.0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7.0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,7.0
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,1,11.0
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,1,0.0
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,1,7.0
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,1,64.0
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,1,15.0


### 대분류 추가

In [29]:
data['bigCate'] = data['testId'].str[2]
data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade,elapsed,bigCate
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.0,6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,3.0,6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,8.0,6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7.0,6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,7.0,6
...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,24.0,3
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,0.0,4
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,11.0,4
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,46.0,4


#### 각 유저의 대분류별 정답률의 평균

In [30]:
user_bigCate_acc = data.groupby(['userID', 'bigCate']).apply(lambda x:x['answerCode'].mean())
user_bigCate_acc

userID  bigCate
0       2          0.722222
        6          0.791908
        8          0.468320
1       4          0.822719
        9          0.903409
                     ...   
7439    4          0.625000
7440    3          0.400000
        5          0.400000
7441    3          0.200000
        4          1.000000
Length: 19739, dtype: float64

In [31]:
user_bigCate_acc[0].index, user_bigCate_acc[0].values

(Index(['2', '6', '8'], dtype='object', name='bigCate'),
 array([0.72222222, 0.79190751, 0.46831956]))

In [32]:
data['user_bigCate_acc'] = 0

In [33]:
for id in tqdm.tqdm(user_id):
    d = data[data['userID']==id]
    cate_unique = d['bigCate'].unique()
    for cate in cate_unique:
        acc_mean = d[d['bigCate']==cate]['answerCode'].mean()
        ind = d[d['bigCate']==cate].index
        data.loc[d[d['bigCate']==cate].index, 'user_bigCate_acc']=acc_mean

100%|██████████| 7442/7442 [01:15<00:00, 98.46it/s] 


In [34]:
data.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade,elapsed,bigCate,user_bigCate_acc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.0,6,0.791908
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,3.0,6,0.791908
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,8.0,6,0.791908
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7.0,6,0.791908
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,7.0,6,0.791908


#### 각 유저의 대분류별 푼 횟수

In [35]:
user_bigCate_count = data.groupby(['userID', 'bigCate']).apply(lambda x:x['answerCode'].count())
user_bigCate_count

userID  bigCate
0       2           36
        6          346
        8          363
1       4          581
        9          352
                  ... 
7439    4           16
7440    3           10
        5            5
7441    3            5
        4            4
Length: 19739, dtype: int64

In [36]:
data['user_bigCate_count'] = 0

In [37]:
for id in tqdm.tqdm(user_id):
    d = data[data['userID']==id]
    cate_unique = d['bigCate'].unique()
    for cate in cate_unique:
        count = d[d['bigCate']==cate]['answerCode'].count()
        ind = d[d['bigCate']==cate].index
        data.loc[d[d['bigCate']==cate].index, 'user_bigCate_count']=count

100%|██████████| 7442/7442 [01:14<00:00, 99.55it/s] 


In [38]:
data.tail(10)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade,elapsed,bigCate,user_bigCate_acc,user_bigCate_count
2526690,7440,A030197005,A030000197,0,2020-10-21 08:33:20,1984,1,3.0,3,0.4,10
2526691,7441,A030071001,A030000071,0,2020-06-05 06:46:41,438,1,0.0,3,0.2,5
2526692,7441,A030071002,A030000071,0,2020-06-05 06:48:43,438,1,122.0,3,0.2,5
2526693,7441,A030071003,A030000071,1,2020-06-05 06:49:23,438,1,40.0,3,0.2,5
2526694,7441,A030071004,A030000071,0,2020-06-05 06:49:57,438,1,34.0,3,0.2,5
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,24.0,3,0.2,5
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,0.0,4,1.0,4
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,11.0,4,1.0,4
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,46.0,4,1.0,4
2526699,7441,A040165004,A040000165,1,2020-08-21 01:08:49,8836,1,73.0,4,1.0,4


#### 각 유저의 대분류별 푼 평균 소요시간

In [39]:
user_bigCate_elapsedTime = data.groupby(['userID', 'bigCate']).apply(lambda x:x['elapsed'].mean())
user_bigCate_elapsedTime

userID  bigCate
0       2          29.972222
        6          31.254335
        8          32.148760
1       4          71.934596
        9          85.409091
                     ...    
7439    4          33.187500
7440    3          23.300000
        5          18.600000
7441    3          44.000000
        4          32.500000
Length: 19739, dtype: float64

In [40]:
data['user_bigCate_elapsedTime'] = 0

In [41]:
for id in tqdm.tqdm(user_id):
    d = data[data['userID']==id]
    cate_unique = d['bigCate'].unique()
    for cate in cate_unique:
        time = d[d['bigCate']==cate]['elapsed'].mean()
        ind = d[d['bigCate']==cate].index
        data.loc[d[d['bigCate']==cate].index, 'user_bigCate_elapsedTime']=time

100%|██████████| 7442/7442 [01:15<00:00, 98.21it/s] 


In [42]:
data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,grade,elapsed,bigCate,user_bigCate_acc,user_bigCate_count,user_bigCate_elapsedTime
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,0.0,6,0.791908,346,31.254335
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,3.0,6,0.791908,346,31.254335
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,8.0,6,0.791908,346,31.254335
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7.0,6,0.791908,346,31.254335
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,7.0,6,0.791908,346,31.254335
...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,24.0,3,0.200000,5,44.000000
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,0.0,4,1.000000,4,32.500000
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,11.0,4,1.000000,4,32.500000
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,46.0,4,1.000000,4,32.500000


In [43]:
# total data 저장
data.to_csv(data_path + 'total_data.csv', index=False)